In [30]:
#!/usr/bin/env python
# coding: utf-8
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import os
import xlwt
import xlrd
from xlutils.copy import copy


def read_article(file_name):
    file = open(file_name, "r")
    filedata = file.readlines()
    article = []
    title = ''
    for eachLine in filedata:
        eachLine = eachLine[:-1]
        if len(eachLine) >= 1:
            if eachLine[:3] != 'URL' and eachLine[:2] != 'ID' and eachLine[:4] != 'Date' and eachLine[:8] != 'Ariticle':
                if eachLine[:5] == 'Title':
                    title += eachLine[8:]
                else:
                    article += eachLine.split(". ")
    sentences = []

    for sentence in article:
#         print('sentence: ', sentence)
        test = sentence.replace("[^a-zA-Z]", " ")
#         print('test: ', test)
        afterToken = test.split(" ")  
#         print('afterToken: ', sentence)
        if len(afterToken) > 5:
            sentences.append(afterToken)
    if len(sentences) > 1:
        sentences.pop()
#     print(sentences)
    return sentences,title

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)
 
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix


def generate_summary(file_name, top_n=5):
    nltk.download("stopwords")
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text anc split it
    sentences,title =  read_article(file_name)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
#     print("Indexes of top ranked_sentence order are ", ranked_sentence)    
    if len(ranked_sentence) < top_n:
        for k in range(len(ranked_sentence)):
            if ranked_sentence[k][1].count('|') < 2:
                summarize_text.append(" ".join(ranked_sentence[k][1]))
    else:
        i = 0
        count = 0
        while count < top_n and i < len(ranked_sentence) :
            if ranked_sentence[i][1].count('|') < 2:
                summarize_text.append(" ".join(ranked_sentence[i][1]))
#                 print(count,': ',summarize_text)
                count += 1
            i += 1
            
    # Step 5 - Offcourse, output the summarize texr
    final_summarize_text = ". ".join(summarize_text)
    if len(final_summarize_text) == 0:
        final_summarize_text += title
#     print("Summarize Text: \n", final_summarize_text)
    return final_summarize_text

# start
booklist = xlrd.open_workbook('PreprocessedNewsList.xls')
sheet = booklist.sheet_by_index(0)
fileNames = (sheet.col_values(0))
directory = "./NewsContents"
os.chdir(directory)  #切换到directory目录
cwd = os.getcwd()  #获取当前目录即dir目录下
print(cwd)
files = os.listdir(cwd)  #列出目录下的文件


newWb = copy(booklist)
newWs = newWb.get_sheet(0);#取sheet表
count = 1
for eachFile in files:
    print(count,': ',eachFile)
    sumText = generate_summary( eachFile, 3)
    print(count,': ',sumText)
#     if count > 2:
#         break
    newWs.write(fileNames.index(eachFile[:-4]),2,sumText)
    count += 1

directory = "../"
os.chdir(directory)  #切换到directory目录
cwd = os.getcwd()  #获取当前目录即dir目录下
print(cwd)
newWb.save('NLTKSumNews.xls')


# sumText = generate_summary( 'FLUT2020050100012.txt', 3)
# print('sum: ', sumText)
# directory = "../"
# os.chdir(directory)  #切换到directory目录
# cwd = os.getcwd()  #获取当前目录即dir目录下
# print(cwd)

/Users/WenhuiJin/Desktop/2020 Spring/CSE519/Test_Summarization-Sentiment_Analyze/FinalProject/NewsContents
1 :  GPHN2020050200011.txt


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


1 :  In Quebec, 373 cases came from the United States and the province reports zero cases connected to travel from China. She said curbs on travel from Iran came before Iran was even reporting it had cases because the public health data in Canada made it clear there was a problem there.. Howard Njoo, said it would have been difficult to close the border in March to the United States or Europe with so few cases in Canada, but the virus was already spreading around the globe.
2 :  HMAP2020042900276.txt
2 :  SF General Boasts Only One Death to Date After Treating 38 COVID-19 Patients In Intensive Care ; ; 
3 :  ALJA2020042900007.txt
3 :  Hoyer said lawmakers will be asked to return when the legislation is complete.. Capitol physician warns lawmakers that it will be at least a year before Congress can return to business as usual.. The physician has privately warned senior lawmakers it will be at least a year before Congress can return to business as usual.
4 :  GPHN2020050300005.txt


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


4 :  In Italy, which has seen the world’s most deaths with more than 28,000, people looked enviously on at Spain as they awaited their own relaxation of restrictions now that the number of new cases has leveled out. Russia sees record spike in coronavirus cases with nearly 10,000 in one day. Russia sees record spike in coronavirus cases with nearly 10,000 in one day
5 :  ALFR2020050100092.txt
5 :  Drawn from the local curriculum, kids have an opportunity to learn via radio classrooms while parents continue home learning with provided materials following the show.. Madam Lago Louise, a refugee now based in Liberia for almost a decade, hopes of having her kids learn in the free government school programme operated by Bridge Liberia seems all but ended.. She's awaiting the local radio programme via the Government run Liberia Broadcasting System to tune to what remains of school, the nation's 'Home Learning Program.'
6 :  GPHN2020050300011.txt


...

6128 :  The death toll at the home appears to be the largest at a long-term care facility in the US, analysts said.. "Geriatricians and experts in long-term care medicine were sounding alarms at the beginning of March and we've essentially been ignored by everyone - federal, state, local government and the nursing home industry," he said.. Sixty-eight veteran residents who tested positive for the virus have died, officials said on Tuesday, and it is not known whether another person who died had COVID-19
6129 :  ALJA2020042900022.txt
6129 :  Presumptive Democratic presidential nominee Joe Biden won the endorsement on Tuesday of Hillary Clinton for the office she fell short of winning in 2016 in her bid to become the first woman elected as president of the United States.. Women favored Clinton over Trump in 2016, exit polls showed, and are expected to play a critical role in swaying the most competitive swing states in the November 3 election between Biden and Trump.. In a statement resp

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords i

6134 :  It was highest at the time of onset and began to decline after a week. The command center said that according to these research findings, when designing prevention and treatment strategies in the future, it will be possible to focus more on the risk interval before onset to early onset of disease, and provide more precise reference for adjustment of epidemic prevention policies to effectively use public health and medical resources.. The command center pointed out that the study found that most of the contact time of infected contacts with index cases fell within one week after the onset of the index case, and it was mainly dominated by living contacts and family members
6135 :  GPHN2020050300008.txt
6135 :  Fewer people in Ontario seeking help for serious heart attacks: Heart and Stroke. The Heart and Stroke Foundation and Canadian Cardiovascular Society are worried that some people may be avoiding hospitals, even when faced with an emergency such as a serious heart attack, du

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/WenhuiJin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


6139 :  "China is a victim, not an accomplice to COVID-19," Vice Foreign Minister Le Yucheng told the NBC, noting that a virus can come and go without any trace and appear in any place in the world.. BEIJING, April 30 (Xinhua) -- Vice Foreign Minister Le Yucheng said China firmly opposes so-called international investigation based on the presumption of guilt regarding the origin of the COVID-19 virus, according to a transcript released on the Foreign Ministry's website Thursday.. China's Vice Foreign Minister Le Yucheng speaks to reporters during an interview in Geneva, Switzerland, Nov
/Users/WenhuiJin/Desktop/2020 Spring/CSE519/Test_Summarization-Sentiment_Analyze/FinalProject
